In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
commsPD = pd.read_sas('commsdata.sas7bdat')

In [8]:
commsPD.head()

,Customer_ID,upsell_xsell,churn,lifetime_value,avg_arpu_3m,acct_age,billing_cycle,nbr_contracts_ltd,credit_class,sales_channel,...,pymts_late_ltd,calls_care_ltd,MB_Data_Usg_M04,MB_Data_Usg_M05,MB_Data_Usg_M06,MB_Data_Usg_M07,MB_Data_Usg_M08,MB_Data_Usg_M09,seconds_of_data_norm,seconds_of_data_log
0,471.0,0.0,0.0,1693.8,44.980000,55.000000,4.0,7.0,b'prime',b'Indirect',...,5.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,4.110874
1,1057.0,0.0,0.0,38682.5,44.989330,47.000000,3.0,5.0,b'prime',b'Indirect',...,5.0,115.0,0.0,0.0,0.0,0.0,330.0,646.0,48300.0,10.785208
2,1177.0,0.0,0.0,5526.4,51.775434,46.176471,7.0,4.8,b'prime',b'Retail',...,5.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,14700.0,9.595671
3,1181.0,0.0,0.0,505.6,51.775434,75.000000,7.0,4.8,b'prime',b'Retail',...,5.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,1185.0,0.0,1.0,9143.2,83.320000,50.000000,5.0,3.0,b'prime',b'Indirect',...,5.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,15840.0,9.670357


In [9]:
pd.__version__

'0.22.0'

In [22]:
# left out "handset_age_grp", for being text
commsPDsel = commsPD[["churn","curr_days_susp","handset_age_grp","ever_days_over_plan","pymts_late_ltd","avg_days_susp","seconds_of_data_norm"]]

In [23]:
commsPDsel.head()

,churn,curr_days_susp,handset_age_grp,ever_days_over_plan,pymts_late_ltd,avg_days_susp,seconds_of_data_norm
0,0.0,1.0,b'< 24 Months',0.0,5.0,0.000000,60.0
1,0.0,0.0,b'< 24 Months',45.0,5.0,4.500000,48300.0
2,0.0,0.0,b'< 24 Months',2.0,5.0,1.000000,14700.0
3,0.0,1.0,b'< 24 Months',0.0,5.0,0.000000,0.0
4,1.0,0.0,b'< 24 Months',13.0,5.0,4.333333,15840.0


In [24]:
# Identify categorical columns
categorical_cols = commsPDsel.select_dtypes(include=['object']).columns.tolist()

In [25]:
print(categorical_cols)

['handset_age_grp']


In [46]:
# Create a OneHotEncoder (dummies)
df_encoded = pd.get_dummies(commsPDsel[categorical_cols])

In [47]:
df_encoded.head()

,handset_age_grp_b'24-48 Month',handset_age_grp_b'< 24 Months',handset_age_grp_b'> 48 Months'
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [48]:
# Concatenate the encoded features with the numerical features
commsPDconc = pd.concat([commsPDsel.drop(categorical_cols, axis=1), df_encoded], axis=1)

In [49]:
# Define features (X) and target variable (y)
X = commsPDsel.drop(['churn','handset_age_grp'], axis=1)
y = commsPDsel['churn']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values using the mean
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict_proba(X_test)[:, 1]

# Evaluate the model (e.g., using AUC)
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

/usr/local/python3/lib/python3.4/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AUC: 0.7813430637908539


In [50]:
# Define features (X) and target variable (y)
X = commsPDconc.drop(['churn'], axis=1)
y = commsPDconc['churn']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values using the mean
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict_proba(X_test)[:, 1]

# Evaluate the model (e.g., using AUC)
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

/usr/local/python3/lib/python3.4/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AUC: 0.8075036106096938


In [54]:
# Train a Gradient Boosting model
modelGB = GradientBoostingClassifier()
modelGB.fit(X_train, y_train)

# Make predictions on the test set
y_pred = modelGB.predict_proba(X_test)[:, 1]

# Evaluate the model (e.g., using AUC)
aucGB = roc_auc_score(y_test, y_pred)
print("AUC:", aucGB)

AUC: 0.8049114062647962


In [ ]:
# Train a Decision Tree model
modelGB = GradientBoostingClassifier()
modelGB.fit(X_train, y_train)

# Make predictions on the test set
y_pred = modelGB.predict_proba(X_test)[:, 1]

# Evaluate the model (e.g., using AUC)
aucGB = roc_auc_score(y_test, y_pred)
print("AUC:", aucGB)